#Robot and Simulator#

[1편 기초편](https://colab.research.google.com/drive/10-LOBTYDF35S9GHNhl9FXcRC4Uwpi9mL)

[3편 Gazebo와 시뮬레이션편](https://colab.research.google.com/drive/1u79f4qnDlLziTdXfjVAfXXKwr5zOFo2r)

##표준적인 ROS메시지 자료형과 정보들##


| 데이터 | 자료형 
|---|---|
영상|sensor_msgs/Image
컴퓨터 비젼 정보|sensor_msgs/CameraInfo

OpenCV 로 부터 받거나 보내는것을 단순화 시키도록 고안된 cv_bridge 패키지라는것이 있다.

|데이터|자료형
|---|---|
Depth Camera 3D 점군 데이터|sensor_msgs/PointCloud2
레이저 스캐너 스캔값|sensor_msgs/LaserScan
축 인코더 주행기록 추정값|geometry_msgs/Transform
로봇팔 인코더 기록값은 보통 |sensor_msgs/JointState 

메시지들로 Broadcast  된다.



##여러가지 시뮬래이터들##

MORSE

VREP

GAZEBO          |   gazebo_ros 패키지를 통해 ROS와 긴밀하게 통신된다.

STAGE

Turtlesim

#원더봇 WanderBot#

첫번째 정리에서 토픽 서비스 액션과 같은 모듈간 통신에 사용되는 개념을 소개했었고

박쥐책의 6장을 보면 sensing 과 actuation 서브 시스템이 소개된다.

여기서는 이 개념들을 합하여 주변을 돌아다닐 수 있는 로봇을 만들어 보자.

##패키지 생성##

먼저 ~/wanderbot_ws에 넣어 둘 작업 공간 디렉터리 트리를 만든다.

    $ mkdir -p ~/wanderbot_ws/src
    
    $ cd ~/wanderbot_ws/src
    
    $ catkin_init_workspace
    
그리고 새 작업 공간에서 패키지를 생성하기 위한 명령어 하나를 실행한다.

wanderbot_ws/src 경로에서

    $ catkin_create_pkg wanderbot rospy geometry_msgs sensor_msgs
    
첫번째 인자 wanderbot은 생성할 새로운 패키지의 이름이다. 이어지는 인자들은 새 패키지가 의존하는 패키지 이름이다.

워크 스페이스가 새롭게 생겼다. 유의하자!

##간단한거 한번 만들어보기##

다음과 같이 wanderbot_ws/src/watderbot/src에 red_light_green_light.py 로 파일 생성해준다.

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

import rospy

from geometry_msgs.msg import Twist

cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)       # (토픽명, 자료형, queue size : 1이면 한개의 메시지만 버퍼링하게함)
                                                                    # 송신노드가 수신노드가 받는거보다 빠르게보내면 rospy가 사이즈 초과 갯수는 폐기한다.
rospy.init_node('red_light_green_light')                            # 노드 실행


red_light_twist = Twist()                                           # Twist 자료형 메시지 생성
                                                                    # 메시지 생성자는 모든 필드를 0으로 채운다. 따라서 멈추라는 명령은 이미 들어감
green_light_twist = Twist()

green_light_twist.linear.x = 0.5                                    # linear.x는 관례적으로 진행 방향을 뜻함. 0.5m/s로 직진하라는 뜻


driving_forward = False                                             # ? initialization같은 느낌인가?

light_change_time = rospy.Time.now()                                # light change time 에 현재 시간 넣어

rate = rospy.Rate(10)                                               # 10hz 송신


while not rospy.is_shutdown():                                      # 뻔한 반복문

    if driving_forward:                                             # driving_forward 가 true 이면
        cmd_vel_pub.publish(green_light_twist)                      # 초록불 켠다 , 지속적으로 보내줘야됨

    else:                                                           # False 이면 (대충)
        cmd_vel_pub.publish(red_light_twist)                        # 빨간불 켠다.

    if light_change_time < rospy.Time.now():                        # 대충 보면 이해 될것임
        driving_forward = not driving_forward                       # True False 토글 느낌인거같음.
        light_change_time = rospy.Time.now() + rospy.Duration(3)

    rate.sleep()                                                    # 10Hz 레이트에 맞게끔 휴식
                                                                        # 이게 없으면 미친듯이 돌아가서 cpu 다먹는다.


그리고 실행 파일로 만들어주자

파일 경로로 이동해서

    $ chmod +x red_light_green_light.py
    
그리고 turttlebot 시뮬레이션 스텍이 설치 되어있는지 확인해 봐야 한다.

    $ sudo apt-get install ros-kinetic-turtlebot-gazebo
    
나는  frontend-lock 이나 lock이 걸리던데 그때는 특정 경로와 락 파일명이 나오게 된다. 

그랬을때 경로를 아래와 같이 경로명에 넣고 뒤에 락파일 종류에 따라 아래와 같이 해주면 된다.

    $ sudo rm 경로명 lock
    
    혹은
    
    $  sudo rm 경로명 frontend-lock
    
을 하여 잠금 파일을 삭제해 준다.

    

###Gazebo 키고 한번 해보자###


    $ roslaunch turtlebot_gazebo turtlebot_world.launch
    
로 초기 터틀봇 세계를 보여주자

해당 워크 스페이스로  가서

    $ source devel/setup.bash
    
    $ rosrun wanderbot red_light_green_light.py cmd_vel:=cmd_vel_mux/input/teleop
    
:= 있는 부분은 cmd_vel을 터틀봇이 기대하는 값으로 name space 를 remapping 하는 부분이다.

하면 인제 터틀봇이 가다 섰다 가다 섰다 해야되는데 왜 안되냐(코드 수정)

됐다~~

##센서 데이터 읽기##

터틀봇은 kinect depth 카메라를 가지고 있으며 터틀봇 소프트웨어는 이를 추출하여

약간의 필터링을 거친후 scan 토픽 상으로 sensor_msgs/LaserScan 메시지를 데이터로 발행한다.

라이다인 양 기만하는 것이다.



가지보 시뮬레이터를 열어놓은 상태에서 다음과 같이 하면 터틀봇 앞의 LaserScan 메시지를 계속 출력한다.

    $ rostopic echo scan
    
재미 없어지면 컨트롤 + C 눌러서 빠져나오자

문자 대부분은 LaserScan 메시지의 ranges 멤버인데, 바로 우리가 관심 있는 부분이다. 

Ranges 배열은 터틀봇으로부터 ranges 배열 색인에서 쉽게 계산 할 수 있는 방향에 있는 가장 가까운 객체까지의 거리를 포함한다.

특히, 메시지 인스턴스가 msg로 명명되어 있다고 가정하면 특정 거리 추정 값을 위한 방향을 아래와 같이 계산할 수 있다.

여기서 i 는 ranges 배열의 색인이다.

    bearing = msg.angle_min + i * msg.angle_max / len(msg.ranges)
    
로봇의 바로 정면에 있는 가장 가까운 장애물까지의 거리를 추출하려면 ranges 배열의 중간 원소를 선택하면 된다.

range_ahead = msg.ranges[len(msg.ranges)/2]

또는 스캐너에 의해서 탐지된 가장 가까운 장애물 까지의 거리를 반환하려면 다음을 실행하자.

closest_range = min(msg.ranges)

(못알아 듣겠어서 그냥 베껴옴)





###장애물까지 거리 출력하는 노드 만들기###

range_ahead.py 라는 파일을 wanderbot_ws/src/wanderbot/src 에 아래와 같이 만들도록 하자

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

import rospy
from sensor_msgs.msg import LaserScan

def scan_callback(msg):
    range_ahead = msg.ranges[len(msg.ranges)/2]
    print "range ahead: %0.1f" % range_ahead

rospy.init_node('range_ahead')

scan_sub = rospy.Subscriber('scan', LaserScan, scan_callback)
rospy.spin()

이제 이정도 코드는 짧고 쉽기때문에 주석을 안넣었다.

이 코드는 가운데에 있는 물체의 거리를 측정하는 노드에 대한 코드이다.

아까 전에 그 움직이는 코드랑 얘랑 같이 실행시켜 보면 터틀봇이 이동하는것에 따라 

장애물과의 거리가 계속 바뀌는것을 볼 수 있다.

로스코어 실행

    $ roscore
    
경로 입력

    $ cd ~/wanderbot_ws
    
    $ source devel/setup.bash

터틀봇 월드 실행

    $ roslaunch turtlebot_gazebo turtlebot_world.launch
    
센서 받기 코드 실행

    $ rosrun wanderbot range_ahead.py
    
빨간불 초록불 코드 실행

    $ rosrun wanderbot red_light_green_light.py cmd_vel:=cmd_vel_mux/input/teleop
    

## Sense and Actuate : Wanderbot##

이전에는 터틀봇을 openloop으로 단순 직진 구동을 하도록 하는 빨간불 초록불 프로그램과 

터틀봇과 가장 가까운 것 과의 거리를 측정하는 range_ahead.py 프로그램을 만들었다.

이러한 두 기능을 합쳐 터틀봇이 0.8m  내의 장애물을 보거나 30초 후에 타임아웃 될 때까지 직진 주행한 다음

정지하고 새로운 방향으로 제자리에서 맴돌도록 하는 wanderbot 프로그램을 만들어보자.

다음과 같은 wander.py 코드를 작성하여 늘 넣던 경로에 넣어놓자.

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

import rospy
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan

def scan_callback(msg):
    global g_range_ahead
    g_range_ahead = min(msg.ranges)


g_range_ahead = 1                                                           # 시작하기위한 임의의 숫자
scan_sub = rospy.Subscriber('scan', LaserScan, scan_callback)
cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)
rospy.init_node('wander')

state_change_time = rospy.Time.now()
driving_forward = True
rate = rospy.Rate(10)

while not rospy.is_shutdown():
    if driving_forward:
        if (g_range_ahead < 0.8 or rospy.Time.now() > state_change_time):
            driving_forward = False
            state_change_time = rospy.Time.now() + rospy.Duration(5)
    else:                                                                   # 전진 주행하지 않음
        if rospy.Time.now() > state_change_time:
            driving_forward = True                                          # 회전 마침! 전진하자
            state_change_time = rospy.Time.now() + rospy.Duration(30)
twist = Twist()
if driving_forward:
    twist.linear.x = 1
else:
    twist.angular.z = 1
cmd_vel_pub.publish(twist)

rate.sleep()

다 나왔었던 느낌의 코드라 주석을 안달았다. 이제 여러분이 한번 달아보자!

#번외#

##터틀봇을 움직여보자##

터틀봇을 움직일때 터틀봇에 실릴 랩탑 컴퓨터와

커맨드를 날려줄 데스크탑 컴퓨터가 필요하다.

두 컴퓨터 모두 ros가 깔려 있어야 하며 랩탑에는 full 버전 말고 그냥 ros가 깔려있어도 된다.

###랩탑에 패키지설치###

    $ ifconfig
    
데스크탑과 랩탑에서 위의 명령을 사용하여  데스크탑과 랩탑의 ip 주소를 알아내자

그리고 랩탑에 다음과 같이 입력한다.

    $ export ROS_MASTER_URI=http://데스크톱아이피:11311
    
    $ export ROS_IP=랩톱아이피
    
그리고 터틀봇 제어를 위한 패키지를 설치하자.

    $ sudo apt-get install ros-kinetic-kobuki
    
    $ sudo apt-get install ros-kinetic-kobuki-core
    
이 패키지를 설치 했다면 SSH를 설치하자 (secure Shell

    $ sudo apt-get install ssh

### 데스크탑에 패키지 설치###

    $ export ROS_MASTER_URI=http://데스크톱아이피:11311
    
    $ export ROS_IP=데스크톱아이피
    
터틀봇 관련 모든 패키지를 설치하자

    $ sudo apt-get install ros-kinetic-kobuki*
    
중간에 몇개 빼먹히는거 같은데 잘 돌아가더라.

데스크탑에도 ssh 설치하자

    $ sudo apt-get install ssh



##터틀봇 구동 테스트##

데스크톱에서 터미널 하나 열어서 roscore 구동해주자

    $ roscore

###SSH를 이용하여 데스크톱에서 랩톱 접속하기###

다른 터미널을 하나 열자

    $ ssh 랩탑유저명@랩탑아이피
    

###터틀봇 포트생성###

방금 접속한 터미널에서 다음과 같이 /dev/kobuki 포트를 생성하자

    $ rosrun kobuki_ftdi create_udev_rules
    
그리고 usb 케이블 뽑았다가 다시 곱고 다음 명령어로 포트를 확인하자

    $ ls -l /dev/kobuki      :띄어쓰기에 주의!
    
lrwxrwxrwx 뭐라뭐라 하면서 맨 뒤에 usb 포트 나온다.

###터틀봇 구동###

    랩탑에 접속된 테이블에서 실행
    
        $ roslaunch kobuki_node minimal.launch
    
    오류 막 뜨면서 뭐라고 징징거리는데, 찾아보니 별 영향 없다더라.

###키보드 제어노드 구동###

다른 터미널을 또 하나 열자

    $ ssh 랩탑유저명@랩탑아이피
    
    $ roslaunch kobuki_keyop keyop.launch
    
그리고 e를 눌러 모터를 활성화 하고 방향키로 터틀봇을 조종할 수 있게 된다.

참고로

    $ roslaunch kobuki_keyop safe_keyop.launch 
    
로 실행하게 되면 범퍼와 절벽감지 휠 드랍 센서를 활성화 시켜 위험한 상황을 피할 수 있게 해준다.

##키넥트와 RVIZ##

우리 터틀봇 위에 얹혀져 있는 키넥트에 대한 ros 관련 드라이버는 다음과 같이 간단히 설치된다.

이 과정들은 ssh 이던 장치에 직접 입력이건 랩탑쪽에서 수행되어야 하는 작업이다.

    $ sudo apt-get install ros-kinetic-openni-camera
    
    $ sudo apt-get install ros-kinetic-openni-launch
    
이후 다음과 같은 명령으로 실행이 가능하다.

    $ roslaunch openni_launch openni.launch
    
그리고 일단은 랩탑에서 

    $ rosrun rviz rviz
    
를 실행시키고

Global Option의 Fixed Frame을 Camera_depth_frame 으로 변경하고

pointcloud2를 추가하여 Topic에 /camera/depth/points 로 하면 점군이 그려지고

depth camera였나 camera였나 추가하고 이것저것 바꿔보면 여러가지 조합이 가능해진다.

## LIDAR(RoboPeak) ##

우선 기본 파일들을 받아야 하는데, 위에서 했던것처럼 중앙 패키지가 아니라 catkin 패키지에 설치된다.

    $ git clone https://github.com/HongDaSeon/rplidar_ros.git
    
받아진 폴더를 내 work space 아무데나, 여기서는 catkin_ws 폴더에 src 폴더 안에 넣는다.

그리고 catkin tool을 설치했고 해당 패키지를 catkin build로 빌드했다면
 
    $ catkin build
    
catkin tool을 설치하지 않았고 catkin_make로 빌드했다면

    $ catkin_make
    
로 빌드 해 준다.

그리고 usb 연결을 한 후 다음과 같이 실행하여 기기를 찾는다.

    $ ls /dev/ttyUSB*
    
그리고 기기에 권한을 부여한다.

    $ sudo chmod 666 /dev/ttyUSB*    ttyUSB* 부분에 해당 기기명을 입력한다.
    
viewer 실행

    $ roslaunch rplidar_ros view_rplidar.launch
    
rviz가 켜지면서 도트가 찍히면 성공

    $ rostopic list
    
    를 통해 /scan 토픽이 발행되는것을 볼 수 있다.

###이 아래는 개인적인 프로젝트 백업###

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

#Something_goes_wrong_or_not.py //old version
import rospy

import numpy as np

from std_msgs.msg import Float32

from sensor_msgs.msg import Image

import time

#범위 설정 총 크기 640width 480 height
left_limit = 100

right_limit = 100

ceiling_limit = 230

floor_limit = 230

is_good = False

over1000per = 1.

def ir_dot_callback(msg):

    cnt = 0
    global over1000per

    start = time.time() 

    ir_level = np.eye(480-ceiling_limit-floor_limit,640-left_limit-right_limit)

    lili = list(msg.data)

    lili = [ord(i) for i in lili]
    
    raw_data = np.array(lili)

    reshaped_data = (raw_data.reshape(480,640,2))

    sliced_data = reshaped_data[ceiling_limit:480-floor_limit,left_limit:640-right_limit,:]

    for r in range(480-ceiling_limit-floor_limit):
        for c in range(640-left_limit-right_limit):
            ir_level[r,c] = (sliced_data[r,c,0])|((sliced_data[r,c,1])<<8)
            if ir_level[r,c] > 1000:
                cnt = cnt + 1

    over1000per = cnt/float((640-left_limit-right_limit)*(480-ceiling_limit-floor_limit))
    is_good = True
    print("time :", time.time() - start) 

    print over1000per

ir_white_pub = rospy.Publisher('ir_white_per', Float32, queue_size=1)    

aquiz_dat = rospy.Subscriber('/camera/ir/image_raw', Image, ir_dot_callback)

rospy.init_node('ir_dot_parse')

rate = rospy.Rate(10)

while ~rospy.is_shutdown():
    findat = Float32()
    findat.data = over1000per
    ir_white_pub.publish(findat)
    rate.sleep()

rospy.spin()

    


In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

#Something_goes_wrong_or_not.py  //threading version

import rospy

import numpy as np

from std_msgs.msg import Float32

from sensor_msgs.msg import Image

import threading

import time

#범위 설정 총 크기 640width 480 height
# left_limit = 150

# right_limit = 150

# ceiling_limit = 100

# floor_limit = 100

is_good = False

over1000per = 1.

global cnt

#가로세로 동일 갯수로 나눠 쓰레드

num = 80

def do_work_line(te,be,le,re):
    for r in range(te,be):
        for c in range(le,re):
            ir_level[r,c] = (reshaped_data[r,c,0])|((reshaped_data[r,c,1])<<8)
            if ir_level[r,c] > 1000:          
                cnt = cnt + 1
            print '=== %d'%count
    
def ir_dot_callback(msg):
    start = time.time()
    global reshaped_data
    global ir_level
    cnt = 0.
    threadss = [] 
    start = time.time() 

    ir_level = np.eye(480,640)

    lili = list(msg.data)

    lili = [ord(i) for i in lili]
    
    raw_data = np.array(lili)

    reshaped_data = (raw_data.reshape(480,640,2))

    for ber in range(num):
        thh = threading.Thread(target=do_work_line, args=((480/num)*ber,(480/num)*(ber+1),(640/num)*ber,(640/num)*(ber+1)))
        thh.start()
        threadss.append(thh)
  
    for thh in threadss:
        thh.join()
    print cnt
    over1000per = cnt/float((640)*(480))
    is_good = True
    print("time :", time.time() - start) 

    print over1000per

### 쓰레드 마지막

rate = rospy.Rate(10)    

ir_white_pub = rospy.Publisher('/ir_white_per', Float32, queue_size=1)    

aquiz_dat = rospy.Subscriber('/camera/ir/image_raw', Image, ir_dot_callback)

rospy.init_node('ir_dot_parse')



while ~rospy.is_shutdown():
    findat = Float32()
    findat.data = over1000per
    ir_white_pub.publish(findat)
    rate.sleep()

rospy.spin()

    


In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

# depth_32_dealing.py


import rospy

import numpy as np

from std_msgs.msg import Float32

from sensor_msgs.msg import Image

import time

import threading

import copy

import struct

global threadss

threadss = []

one_flag = False

def parsing(msg):
    global raw_data
    
    global pack

    global sh_depth

    sh_depth = np.eye(480,640)

    start = time.time() 

    pack = copy.copy(msg)

    lili = list(msg.data)
    #print lili
    # lili = [l.replace('', '\x00') for l in lili]
    #print lili
    raw_data = np.array(lili).reshape(480,640,4)

    #print raw_data

    num = 80
         
    for ber in range(num):

        thh = threading.Thread(target=parell_process, args=((480/num)*ber,(480/num)*(ber+1),(640/num)*ber,(640/num)*(ber+1)))

        thh.start()

        threadss.append(thh)
  
    for thh in threadss:

        thh.join()
    
    one_flag = True

    # print 'data = %f'%sh_depth[100,100]

    print("time :", time.time() - start) 

    #print pack.data[1000]

    

def parell_process(te,be,le,re):
    
    for r in range(te, be):

        for c in range(le, re):
            for d in range(4):
                if raw_data[r,c,d] == b'': 
                    raw_data[r,c,d] = '\x00' 
                    print '+++++++++++++++++++++++ : %s'%raw_data[r,c,d]
            sh_depth[r,c] = (struct.unpack('f',raw_data[r,c,0]+raw_data[r,c,1]+raw_data[r,c,2]+raw_data[r,c,3]))[0]
            #print r, c
            #print 'nowdepth = %.3f'%sh_depth[r,c]

            pack.data = list(sh_depth.reshape(480,640))
    

single_byte_depth = rospy.Publisher('comped_depth', Image, queue_size=1)

aquiz_dat = rospy.Subscriber('/camera/depth/image_raw', Image, parsing)

rospy.init_node('comp_depth')

rate = rospy.Rate(5)

while ~rospy.is_shutdown() and one_flag == True:

    pack.encoding = 'None'

    single_byte_depth.publish(pack)

    rate.sleep()

rospy.spin()

    


In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

# turtle_wander_test.py //old version

import rospy
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from kobuki_msgs.msg import BumperEvent

def scan_callback(msg):
    global whitness
    whitness = msg.data

def bumper_callback(msg):
    global kung
    kung = msg.state
    where = msg.bumper

whitness = 1                                                           # 시작하기위한 임의의 숫자
kung = 0
where = 1
scan_sub = rospy.Subscriber('ir_white_per', Float32, scan_callback)
bump_sub = rospy.Subscriber('/mobile_base/events/bumper', BumperEvent, bumper_callback)
cmd_vel_pub = rospy.Publisher('/mobile_base/commands/velocity', Twist, queue_size=1)
rospy.init_node('t_wander')

state_change_time = rospy.Time.now()
driving_forward = False
rate = rospy.Rate(50)

while not rospy.is_shutdown():
    if driving_forward:
        if (whitness > 0.1 or rospy.Time.now() > state_change_time):
            driving_forward = False
            state_change_time = rospy.Time.now() + rospy.Duration(5)
            where = 4
    else:                                                                   # 전진 주행하지 않음
        if rospy.Time.now() > state_change_time and whitness < 0.1 and kung == 0:
            driving_forward = True                                          # 회전 마침! 전진하자
            state_change_time = rospy.Time.now() + rospy.Duration(30)

    twist = Twist()

    if driving_forward:
        twist.linear.x = 0.1
    else:
        if where == 0:
            twist.linear.x = -0.05
            twist.angular.z = -0.3
        elif where == 1:
            twist.linear.x = -0.05
            twist.angular.z = 0.3
        elif where == 2:
            twist.linear.x = -0.05
            twist.angular.z = 0.3
        else:
            twist.angular.z = -0.3
        
    cmd_vel_pub.publish(twist)

    rate.sleep()

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

# turtle_wander_test.py

import rospy
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from kobuki_msgs.msg import BumperEvent
 
def scan_callback(msg):
    global whitness
    whitness = msg.data
 
def bumper_callback(msg):
    global kung
    kung = msg.state
    where = msg.bumper
 
 
whitness = 1                                                           # 시작하기위한 임의의 숫자
kung = 0
where = 1
scan_sub = rospy.Subscriber('ir_white_per', Float32, scan_callback)
bump_sub = rospy.Subscriber('/mobile_base/events/bumper', BumperEvent, bumper_callback)
cmd_vel_pub = rospy.Publisher('/mobile_base/commands/velocity', Twist, queue_size=1)
rospy.init_node('t_wander')
 
state_change_time = rospy.Time.now()
driving_forward = False
rate = rospy.Rate(10)
 
while not rospy.is_shutdown():
    if driving_forward:
        if (whitness > 0.1 or whitness ==0.0 or kung == 1 or rospy.Time.now() > state_change_time):
            driving_forward = False
            state_change_time = rospy.Time.now() + rospy.Duration(5)
            where = 4
    else:                                                                   # 전진 주행하지 않음
        if (rospy.Time.now() > state_change_time and whitness < 0.1 and kung == 0):
            driving_forward = True                                          # 회전 마침! 전진하자
            state_change_time = rospy.Time.now() + rospy.Duration(5)

    twist = Twist()
 
    if driving_forward:
        twist.linear.x = 0.15
        twist.angular.z = 0.0
    else:
        if where == 0:
            twist.linear.x = 0.0
            twist.angular.z = -0.3
        elif where == 1:
            twist.linear.x = 0.0
            twist.angular.z = 0.3
        elif where == 2:
            twist.linear.x = 0.0
            twist.angular.z = 0.3
        else:
            twist.linear.x = 0.0
            twist.angular.z = -0.3
        
    cmd_vel_pub.publish(twist)
 
    rate.sleep()

#Chapter 8 키보드 구동부분 Jump#

    

#World Map 만들기#

ROS의 navigation map은 2차원 격자로 표현된다. 흰색은 열린공간, 검은색은 점유된것, 회색은 알지 못하는 것을 나타낸다.

이는 이미지 파일로 저장되는데 PNG JPG PGM 과 같은 다양한 파일 형식이 지원된다.

컬러이미지도 사용 가능하나 ROS에 의해 해독되기 전에  grayscale로 변환된다.

격자 resolution(m단위), 지도 원점 위치, 셀 점유 여부를 결정하는데 사용되는 임계값과 같은 부가적인 정보를 가지는 YAML 파일이

각 지도와 연관되어있다.

##간략한 설명##

map.yaml 파일은 다음과 같은 구조를 가지고있다.

    image: map.pgm
    resolution : 0.1
    origin: [0.0, 0.0, 0.0]
    occupied_thresh: 0.65
    free_thresh: 0.196
    negate: 1

이미지와 지도는 서로 다른 좌표 프레임 및 연관된 규칙을 가진다

이미지는 y출이 아래로 내려가며 왼쪽 위에서부터 색인되며 보통 0에서 255까지 정수값을 저장한다

255로 갈수록 흰색이고 0으로 갈수록 검은색이다.

한편 지도는 YAML 파일에 지정된 임의의 원점을 가질 수 있다.

지도는 세계에 대한 확률적인 표현으로 

큰값은 무엇인가가 있음을 뜻하고

작은값은 빈 공간을 뜻한다.



이 지도는 10cm^2 을 나타내는 셀로 세계를 표현하며, 원점은 0,0,0 이다. 

셀은 그 값이 이미지 형식에서 허용되는 전체 범위의 65% 이상이면 점유된 것으로 간주된다.

셀이 허용된 범위의 19.6%보다 적은 값을 가지면 비어있는 것으로 간주한다.

이는 점유된 셀이 큰 값을 가지며 이미지에서 밝은색으로 나타남을 뜻한다. 

하지만 비어있는 공간이 흰색으로 보이는것이 보다 직관적이므로

ROS에서 사용되기 전에 negate flag 는 셀의 값이 역으로 되도록 만든다.

    49 (255*0.196 = 49.98) 보다 적은값은 비어있는것으로 간주
    
    165 (255*0.65 = 165.75) 보다 큰 값은 점유된 것으로 간주.
    
다른 모든 셀은 미지의 것으로 분류된다.

지도는 이미지 파일로 표현되고 다른 이미지 편집기로 편집 가능하다

    $ sudo apt-get install gimp
    
로 우분투 이미지 편집툴을 설치해보자

    $ gimp
    
로 간단히 실행 가능하다.

생각보다 쓸만하다.

보통 지도를 통해 가짜 장애물을 대충 그려서 추가하곤 하는데, 이러한 방법은 로봇이 특정 지역을 통해 가는 경로를

계획하지 못하도록 하는 것이다.


##rosbag 을 이용한 데이터 기록##

rosbag은 메시지를 기록하고 나중에 재생하게 해주는 도구이다.

이는 새로운 알고리즘을 디버깅할 때 유용하다.

rosbag은 데이터를 기록하고 재생할 뿐만 아니라 다른 일들을 할 수 있다.

그런데 지금은 데이터 기록과 재생만 다룬다고 하신다.

메시지를 기록하기 위해서는 record 기능과 토픽 이름 목록을 사용한다.

예를들어

    $ rosbag record scan tf
    
를 실행하면 scan 토픽과 tf 토픽을 기록할 수 있다.

기본적으로는 YYYY-MM-DD-HH-mm-ss.bag 의 이름으로 저장되지만 바꿀 수도 있다.

    $ rosbag record -O foo.bag scan tf
    
를 실행하면 foo.bag 이라는 파일명으로 저장된다.

    $ rosbag record -o foo scan tf
    
를 실행하면 foo_YYYY-MM-DD-HH-mm-ss.bag 으로 접두사가 붙는다.

또한

    $ rosbag record -a
    
를 이용하여 현재 발행되는 모든 토픽을 기록할 수도 있다.



play 기능을 사용해서 이미 기록된 메시지 모음 파일을 재생할 수 있다.

    $ rosbag play --clock foo.bag
    
이는 기록된 메시지를 ROS 메시지가 노드에서 실시간으로 생성되는것 처럼 재생할 것이다.
    
--clock 플래그는 rosbag이 세계 시간을 발행하게 되고 지도를 구축할때 중요할 것이라고한다.

주의할점은 --clock을 사용하면 세계시간을 받아오는데, 문제는 Gazebo와 같은 시뮬레이터에서도

이와같은 세계시간을 발행하게 된다는 것이다.

그러면 이시간 저시간 엎치락 뒤치락 하면서

토픽이 시간이 앞뒤로 변경되는것처럼 발행된다.

가장 쉬운 해결방법은 시뮬레이터를 끄는것이다.

    $ rosbag info laser.bag
    
을 이용하여 laser.bag 메시지 모음파일에 있는 정보를 볼 수 있다.    

더 많은 정보는 [요기](http://wiki.ros.org/rosbag?distro=kinetic)  (kinetic이 있는지는 모르겟다) 에서 찾을 수 있다.

##지도구축 G-mapping##

#SLAM 지도구축 RtabMap#

설치

    $ sudo apt-get install ros-kinetic-rtabmap-ros
       
실행

키넥트 드라이버(openni) 설치는 저 위에 키넥트 부분에 있음

이건 실행

    $ roslaunch openni_launch openni.launch depth_registration:=true

mapping

    $ roslaunch rtabmap_ros rtabmap.launch rtabmap_args:="--delete_db_on_start"
    

 [자세한 정보](http://wiki.ros.org/rtabmap_ros)
 
 [소스](https://github.com/HongDaSeon/rtabmap_ros)

#Install MAVROS#

ros가 catkin tools를 사용할수 있게 해야 한다. 아래를 입력해 설치하자

    $ sudo apt-get install python-catkin-tools
    
    $ sudo apt-get install python-wstool
    
    $ sudo apt-get install python-rosinstall-generator
    
</br>
    
그리고 워크스페이스 초기화 하자

    $ mkdir -p ~/catkin_ws/src
    
    $ cd ~/catkin_ws
    
    $ catkin init
</br>
wstool 을 이전에 사용한적이 없다면 아래와 같이 타이핑

    $ wstool init ~/catkin_ws/src
    
</br>

아래와 같이 최신버전 혹은 stabilized 버전 둘중 하나를 설치(설치할 패키지 워크스페이스에 들어가서)

    $ rosinstall_generator --upstream mavros | tee /tmp/mavros.rosinstall             //stabilized
    
    $ rosinstall_generator --upstream-development mavros | tee /tmp/mavros.rosinstall            //latest
    
그리고 이거실행

    $ rosinstall_generator mavlink | tee -a /tmp/mavros.rosinstall
   
마브로스가 설치되어야 하는 패키지에 들어가서 아래와 같은 명령으로 설치한다.
   
    $ wstool merge -t src /tmp/mavros.rosinstall
    
    $ wstool update -t src
    
    $ rosdep install --from-paths src --ignore-src --rosdistro kinetic -y
    
    $ catkin build
    
    
catkin_make이 이미 실행되어서 뭐가 안된다고 하면 src폴더빼고 다 지워(필요가 없을시, 아니면 새로운 패키지를 만드는게 방법)
 
 [자세한 정보](http://wiki.ros.org/mavros)
 
 
 
[SITL python 예제](https://github.com/HongDaSeon/Firmware/tree/master/integrationtests/python_src/px4_it/mavros)

#MAVROS 노드를 열어보자!#

geographiclib 설치

    $ sudo bash ~/MAVtest_ws/build/mavros/catkin_generated/installspace/install_geographiclib_datasets.sh
    
px4 실행? 얘는 뭔지 몰겟고

    $ roslaunch mavros px4.launch 
    

QGC에서 baudrate 설정 해주고

해당 포트의 프로토콜을 MAVLINK2.0으로 바꿔주자!

장치주소 얻기

    $ ls /dev/ttyUSB*
    
권한주기

    $ sudo chmod 666 /dev/ttyUSB[]      //[]에 숫자 ACM일수도 있음

    $ rosrun mavros mavros_node _fcu_url:="/dev/장치주소:Baudrate"    // 띄어쓰기에 주의

    eg. $ rosrun mavros mavros_node _fcu_url:="/dev/ttyACM0:115200"   // micro 5핀 연결했을때 115200이 거의 최대
    
    eg. $ rosrun mavros mavros_node _fcu_url:="/dev/ttyUSB0:921600"   // TTL2USB로 연결했을때 921600 을 대부분 사용
    
하면 인제 통신 열림

(참고 : 에러 무시)

근데 보통 QGC에서 telem2 를 921600 보레이트 설정해 주고 빠르게 주고받는거같음

불안하게 너무 쉬운뎅?

#Obstacle Avoidance#

[Github page](https://github.com/PX4/avoidance#obstacle-detection-and-avoidance)

이대로 하면 되는데

angular_velocity_stdev: !degrees 0.02

이런 에러가 나면 다음과 같이 해주자

    $ sudo pip uninstall PyYAML
    
    $ sudo pip install PyYAML==3.13
    
catkin_ws에 마브로스랑 이거랑 다 설치하는걸 권장

launch 하기전에 

    $ export ROS_PACKAGE_PATH=${ROS_PACKAGE_PATH}:~/Firmware
    
해주자

글로벌 플래너 실행

    $ roslaunch global_planner global_planner_stereo.launch
    
OFFBOARD MODE

    $ rosrun mavros mavsys mode -c OFFBOARD

ARM

    $ rosrun mavros mavsafety arm